In [ ]:
!pip install geopandas

     |████████████████████████████████| 994 kB 4.9 MB/s 
     |████████████████████████████████| 15.4 MB 36 kB/s 
     |████████████████████████████████| 6.3 MB 15.9 MB/s 


In [ ]:
# Bibliotecas
import pandas as pd
import numpy as np

In [ ]:
# Download e tratamento inicial dos dados
data = pd.read_csv('https://covid19.ssp.df.gov.br/resources/dados/dados-abertos.csv?param=[random]', 
                   sep=';',dtype={'Data':str, 'Data Cadastro':str, 'Sexo':str, 'Faixa Etária':str, 
                                  "Idade":str, 'RA':str, 'UF':str,'Óbito':str, 'dataPrimeirosintomas':str,
                                  'Pneumopatia':str, 'Doença Hematológica':str, 'Distúrbios Metabólicos':str, 
                                  'Imunopressão':str, 'Obesidade':str, 'Outros':str, 
                                  'Cardiovasculopatia':str, 'Nefropatia':str})
# Deletando colunas que não serão utilizadas
data = data.drop(['UF', 'Pneumopatia', 'Doença Hematológica', 'Distúrbios Metabólicos', 
                  'Imunopressão', 'Obesidade', 'Outros', 'Cardiovasculopatia', 'Nefropatia'], axis=1)
data['Data Cadastro'] = data['Data Cadastro'].replace(['2020-03-18','2020-05-06','2020-06-06',
                                                       '2020-07-06','2020-08-06'],
                                                      ['18/03/2020','06/05/2020','06/06/2020','06/07/2020','06/08/2020'])
# Preenchendo vazios
data['Data Cadastro'] = data['Data Cadastro'].fillna(data['dataPrimeirosintomas'])

# Formatando colunas para formato datetime
data['Data'] = pd.to_datetime(data['Data'], format='%d/%m/%Y')
data['Data Cadastro'] = pd.to_datetime(data['Data Cadastro'], format='%d/%m/%Y')
data['dataPrimeirosintomas'] = pd.to_datetime(data['dataPrimeirosintomas'], format='%d/%m/%Y')

In [ ]:
data.count()

Data                    482337
Data Cadastro           482337
Sexo                    482337
Faixa Etária            482337
RA                      462971
Óbito                   482337
Data do Óbito            10245
dataPrimeirosintomas    482337
Idade                   482337
dtype: int64

In [ ]:
# Preenchendo vazios
data['RA'] = data['RA'].fillna('Não Informado')

data.head()

,Data,Data Cadastro,Sexo,Faixa Etária,RA,Óbito,Data do Óbito,dataPrimeirosintomas,Idade
0,2021-09-15,2020-03-18,Masculino,50 a 59 anos,Plano Piloto,Não,NaN,2020-04-23,54
1,2021-09-15,2020-03-18,Masculino,50 a 59 anos,Plano Piloto,Não,NaN,2020-08-03,52
2,2021-09-15,2020-03-18,Masculino,40 a 49 anos,Lago Sul,Não,NaN,2020-03-09,47
3,2021-09-15,2020-03-18,Masculino,>= 60 anos,Águas Claras,Não,NaN,2020-03-14,63
4,2021-09-15,2020-03-18,Feminino,20 a 29 anos,Plano Piloto,Não,NaN,2020-03-14,29


In [ ]:
# Criando coluna de casos
data['casos'] = 1

# Criando coluna numérica para óbitos
data = data.replace({'Óbito': r'^Si.$'}, {'Óbito': r'1'}, regex=True)
data = data.replace({'Óbito': r'^Nã.$'}, {'Óbito': r'0'}, regex=True)

data['Óbito'] = pd.to_numeric(data['Óbito'])

# Substituindo campos vazios, que não são NaNs
data['RA'] = data['RA'].replace({" ": "Não Informado"})

In [ ]:
data.head()

,Data,Data Cadastro,Sexo,Faixa Etária,RA,Óbito,Data do Óbito,dataPrimeirosintomas,Idade,casos
0,2021-09-15,2020-03-18,Masculino,50 a 59 anos,Plano Piloto,0,NaN,2020-04-23,54,1
1,2021-09-15,2020-03-18,Masculino,50 a 59 anos,Plano Piloto,0,NaN,2020-08-03,52,1
2,2021-09-15,2020-03-18,Masculino,40 a 49 anos,Lago Sul,0,NaN,2020-03-09,47,1
3,2021-09-15,2020-03-18,Masculino,>= 60 anos,Águas Claras,0,NaN,2020-03-14,63,1
4,2021-09-15,2020-03-18,Feminino,20 a 29 anos,Plano Piloto,0,NaN,2020-03-14,29,1


In [ ]:
# Consultando óbitos
data.query("Óbito==1")

,Data,Data Cadastro,Sexo,Faixa Etária,RA,Óbito,Data do Óbito,dataPrimeirosintomas,Idade,casos
244,2021-09-15,2020-03-28,Masculino,>= 60 anos,Guará,1,01/04/2020,2020-03-24,83,1
272,2021-09-15,2020-03-29,Masculino,>= 60 anos,Plano Piloto,1,03/04/2020,2020-03-26,68,1
274,2021-09-15,2020-03-29,Feminino,>= 60 anos,Sudoeste/Octogonal,1,23/03/2020,2020-03-19,62,1
284,2021-09-15,2020-03-29,Masculino,>= 60 anos,Jardim Botânico,1,31/03/2020,2020-03-26,74,1
285,2021-09-15,2020-03-29,Feminino,>= 60 anos,Jardim Botânico,1,09/04/2020,2020-03-13,78,1
...,...,...,...,...,...,...,...,...,...,...
480800,2021-09-15,2021-09-13,Masculino,>= 60 anos,Sobradinho,1,14/09/2021,2021-09-12,88,1
480916,2021-09-15,2021-09-14,Masculino,>= 60 anos,Arniqueira,1,13/09/2021,2021-08-21,72,1
480917,2021-09-15,2021-09-14,Feminino,40 a 49 anos,Gama,1,12/09/2021,2021-08-31,41,1
481054,2021-09-15,2021-09-14,Masculino,>= 60 anos,Santa Maria,1,14/09/2021,2021-09-13,73,1


In [ ]:
data.query('RA == " "') 

,Data,Data Cadastro,Sexo,Faixa Etária,RA,Óbito,Data do Óbito,dataPrimeirosintomas,Idade,casos


In [ ]:
# Ordenando por RA e data
d2 = data.sort_values(['RA', 'Data Cadastro'], ascending=[False, True])
d2.head()

,Data,Data Cadastro,Sexo,Faixa Etária,RA,Óbito,Data do Óbito,dataPrimeirosintomas,Idade,casos
3,2021-09-15,2020-03-18,Masculino,>= 60 anos,Águas Claras,0,NaN,2020-03-14,63,1
11,2021-09-15,2020-03-18,Masculino,30 a 39 anos,Águas Claras,0,NaN,2020-03-15,31,1
15,2021-09-15,2020-03-18,Feminino,40 a 49 anos,Águas Claras,0,NaN,2020-03-13,46,1
39,2021-09-15,2020-03-19,Masculino,40 a 49 anos,Águas Claras,0,NaN,2020-03-09,47,1
51,2021-09-15,2020-03-19,Feminino,30 a 39 anos,Águas Claras,0,NaN,2020-03-17,31,1


In [ ]:
#d2['casos_csum'] = d2.groupby(['RA'])['casos'].cumsum()

In [ ]:
d2.head()

,Data,Data Cadastro,Sexo,Faixa Etária,RA,Óbito,Data do Óbito,dataPrimeirosintomas,Idade,casos
3,2021-09-15,2020-03-18,Masculino,>= 60 anos,Águas Claras,0,NaN,2020-03-14,63,1
11,2021-09-15,2020-03-18,Masculino,30 a 39 anos,Águas Claras,0,NaN,2020-03-15,31,1
15,2021-09-15,2020-03-18,Feminino,40 a 49 anos,Águas Claras,0,NaN,2020-03-13,46,1
39,2021-09-15,2020-03-19,Masculino,40 a 49 anos,Águas Claras,0,NaN,2020-03-09,47,1
51,2021-09-15,2020-03-19,Feminino,30 a 39 anos,Águas Claras,0,NaN,2020-03-17,31,1


In [ ]:
d2['Óbito'].astype(int)
#d2['obitos_csum'] = d2.groupby(['RA'])['Óbito'].cumsum()

3         0
11        0
15        0
39        0
51        0
         ..
481236    0
481335    0
481918    0
481945    0
481953    0
Name: Óbito, Length: 482337, dtype: int64

In [ ]:
d2.head()

,Data,Data Cadastro,Sexo,Faixa Etária,RA,Óbito,Data do Óbito,dataPrimeirosintomas,Idade,casos
3,2021-09-15,2020-03-18,Masculino,>= 60 anos,Águas Claras,0,NaN,2020-03-14,63,1
11,2021-09-15,2020-03-18,Masculino,30 a 39 anos,Águas Claras,0,NaN,2020-03-15,31,1
15,2021-09-15,2020-03-18,Feminino,40 a 49 anos,Águas Claras,0,NaN,2020-03-13,46,1
39,2021-09-15,2020-03-19,Masculino,40 a 49 anos,Águas Claras,0,NaN,2020-03-09,47,1
51,2021-09-15,2020-03-19,Feminino,30 a 39 anos,Águas Claras,0,NaN,2020-03-17,31,1


In [ ]:
# Calculando diferença de datas para a criação da coluna 'recuperado'
d2['delta']= d2['Data'] - d2['dataPrimeirosintomas']
d2['delta']=pd.to_numeric(d2['delta'].dt.days, downcast='integer')

In [ ]:
d2.head()

,Data,Data Cadastro,Sexo,Faixa Etária,RA,Óbito,Data do Óbito,dataPrimeirosintomas,Idade,casos,delta
3,2021-09-15,2020-03-18,Masculino,>= 60 anos,Águas Claras,0,NaN,2020-03-14,63,1,550
11,2021-09-15,2020-03-18,Masculino,30 a 39 anos,Águas Claras,0,NaN,2020-03-15,31,1,549
15,2021-09-15,2020-03-18,Feminino,40 a 49 anos,Águas Claras,0,NaN,2020-03-13,46,1,551
39,2021-09-15,2020-03-19,Masculino,40 a 49 anos,Águas Claras,0,NaN,2020-03-09,47,1,555
51,2021-09-15,2020-03-19,Feminino,30 a 39 anos,Águas Claras,0,NaN,2020-03-17,31,1,547


In [ ]:
# Coluna 'recuperado'
d2['recuperado'] = np.where(np.logical_and(d2['delta'].values >= 15, d2['Óbito'].values == 0),1,0)


In [ ]:
# Soma dos recuperados
d2['recuperado'].sum()

464500

In [ ]:
d2.head()

,Data,Data Cadastro,Sexo,Faixa Etária,RA,Óbito,Data do Óbito,dataPrimeirosintomas,Idade,casos,delta,recuperado
3,2021-09-15,2020-03-18,Masculino,>= 60 anos,Águas Claras,0,NaN,2020-03-14,63,1,550,1
11,2021-09-15,2020-03-18,Masculino,30 a 39 anos,Águas Claras,0,NaN,2020-03-15,31,1,549,1
15,2021-09-15,2020-03-18,Feminino,40 a 49 anos,Águas Claras,0,NaN,2020-03-13,46,1,551,1
39,2021-09-15,2020-03-19,Masculino,40 a 49 anos,Águas Claras,0,NaN,2020-03-09,47,1,555,1
51,2021-09-15,2020-03-19,Feminino,30 a 39 anos,Águas Claras,0,NaN,2020-03-17,31,1,547,1


In [ ]:
# Ordenando por RA e data decrescente
d2 =  d2.sort_values(['RA', 'Data Cadastro'], ascending=[False, False])

In [ ]:
#d2['mmo_obitos'] = d2.groupby(['RA'])['obitos_csum'].rolling(7).mean().reset_index(0,drop=True)

In [ ]:
d2.head(10)

,Data,Data Cadastro,Sexo,Faixa Etária,RA,Óbito,Data do Óbito,dataPrimeirosintomas,Idade,casos,delta,recuperado
481696,2021-09-15,2021-09-15,Masculino,40 a 49 anos,Águas Claras,0,NaN,2021-09-14,41,1,1,0
481887,2021-09-15,2021-09-15,Feminino,40 a 49 anos,Águas Claras,0,NaN,2021-09-13,42,1,2,0
481897,2021-09-15,2021-09-15,Masculino,40 a 49 anos,Águas Claras,0,NaN,2021-09-13,40,1,2,0
481912,2021-09-15,2021-09-15,Feminino,30 a 39 anos,Águas Claras,0,NaN,2021-09-13,37,1,2,0
481922,2021-09-15,2021-09-15,Masculino,30 a 39 anos,Águas Claras,0,NaN,2021-09-13,33,1,2,0
481927,2021-09-15,2021-09-15,Feminino,30 a 39 anos,Águas Claras,0,NaN,2021-09-13,37,1,2,0
481933,2021-09-15,2021-09-15,Masculino,20 a 29 anos,Águas Claras,0,NaN,2021-09-13,28,1,2,0
481934,2021-09-15,2021-09-15,Feminino,40 a 49 anos,Águas Claras,0,NaN,2021-09-13,40,1,2,0
481938,2021-09-15,2021-09-15,Masculino,30 a 39 anos,Águas Claras,0,NaN,2021-09-13,36,1,2,0
481978,2021-09-15,2021-09-15,Feminino,<= 19 anos,Águas Claras,0,NaN,2021-09-13,19,1,2,0


In [ ]:
#d2['mmo_casos'] = d2['casos_csum'].rolling(7).mean().reset_index(0,drop=True)
d2.head()

,Data,Data Cadastro,Sexo,Faixa Etária,RA,Óbito,Data do Óbito,dataPrimeirosintomas,Idade,casos,delta,recuperado
481696,2021-09-15,2021-09-15,Masculino,40 a 49 anos,Águas Claras,0,NaN,2021-09-14,41,1,1,0
481887,2021-09-15,2021-09-15,Feminino,40 a 49 anos,Águas Claras,0,NaN,2021-09-13,42,1,2,0
481897,2021-09-15,2021-09-15,Masculino,40 a 49 anos,Águas Claras,0,NaN,2021-09-13,40,1,2,0
481912,2021-09-15,2021-09-15,Feminino,30 a 39 anos,Águas Claras,0,NaN,2021-09-13,37,1,2,0
481922,2021-09-15,2021-09-15,Masculino,30 a 39 anos,Águas Claras,0,NaN,2021-09-13,33,1,2,0


In [ ]:
d2.count()

Data                    482337
Data Cadastro           482337
Sexo                    482337
Faixa Etária            482337
RA                      482337
Óbito                   482337
Data do Óbito            10245
dataPrimeirosintomas    482337
Idade                   482337
casos                   482337
delta                   482337
recuperado              482337
dtype: int64

In [ ]:
# Deletando colunas
d2 = d2.drop(['Data', 'dataPrimeirosintomas','Idade', 'delta'], axis=1)

In [ ]:
# Criando coluna para join espacial
d2['ra'] = d2['RA']

d2 = d2.drop(['RA'], axis=1)

In [ ]:
import geopandas as gpd

# Dado geográfico das RAs
ra = gpd.read_file('RA.shp', encoding='UTF-8')
ra.head()

,ra,geometry
0,Plano Piloto,POINT (182864.551 8260433.709)
1,Gama,POINT (163796.322 8228531.595)
2,Brazlândia,POINT (163814.577 8269927.154)
3,Sobradinho,POINT (201689.520 8269503.396)
4,Planaltina,POINT (228573.880 8265867.938)


In [ ]:
# Join espacial
join = ra.merge(d2, on ='ra')

In [ ]:
# Projeção do join
join.crs

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
# Modificando projeção para sistema de coordenadas geográficas
join = join.to_crs('EPSG:4326')

In [ ]:
join.head()

,ra,geometry,Data Cadastro,Sexo,Faixa Etária,Óbito,Data do Óbito,casos,recuperado
0,Plano Piloto,POINT (-47.95882 -15.71442),2021-09-15,Masculino,20 a 29 anos,0,NaN,1,0
1,Plano Piloto,POINT (-47.95882 -15.71442),2021-09-15,Masculino,>= 60 anos,0,NaN,1,0
2,Plano Piloto,POINT (-47.95882 -15.71442),2021-09-15,Masculino,>= 60 anos,0,NaN,1,0
3,Plano Piloto,POINT (-47.95882 -15.71442),2021-09-15,Feminino,40 a 49 anos,0,NaN,1,0
4,Plano Piloto,POINT (-47.95882 -15.71442),2021-09-15,Feminino,50 a 59 anos,0,NaN,1,0


In [ ]:
df = join.drop('geometry', axis=1) 

In [ ]:
df.head()

,ra,Data Cadastro,Sexo,Faixa Etária,Óbito,Data do Óbito,casos,recuperado
0,Plano Piloto,2021-09-15,Masculino,20 a 29 anos,0,NaN,1,0
1,Plano Piloto,2021-09-15,Masculino,>= 60 anos,0,NaN,1,0
2,Plano Piloto,2021-09-15,Masculino,>= 60 anos,0,NaN,1,0
3,Plano Piloto,2021-09-15,Feminino,40 a 49 anos,0,NaN,1,0
4,Plano Piloto,2021-09-15,Feminino,50 a 59 anos,0,NaN,1,0


In [ ]:
# Criando coluna de coordenadas
df['geo'] = join.geometry.y.astype(str) +','+ join.geometry.x.astype(str)

df.head()

,ra,Data Cadastro,Sexo,Faixa Etária,Óbito,Data do Óbito,casos,recuperado,geo
0,Plano Piloto,2021-09-15,Masculino,20 a 29 anos,0,NaN,1,0,"-15.714424616292403,-47.9588182845936"
1,Plano Piloto,2021-09-15,Masculino,>= 60 anos,0,NaN,1,0,"-15.714424616292403,-47.9588182845936"
2,Plano Piloto,2021-09-15,Masculino,>= 60 anos,0,NaN,1,0,"-15.714424616292403,-47.9588182845936"
3,Plano Piloto,2021-09-15,Feminino,40 a 49 anos,0,NaN,1,0,"-15.714424616292403,-47.9588182845936"
4,Plano Piloto,2021-09-15,Feminino,50 a 59 anos,0,NaN,1,0,"-15.714424616292403,-47.9588182845936"


In [ ]:
#df = df.drop(['x', 'y'], axis=1)
#df.head()

In [ ]:
df.count

<bound method DataFrame.count of                    ra  ...                                    geo
0        Plano Piloto  ...  -15.714424616292403,-47.9588182845936
1        Plano Piloto  ...  -15.714424616292403,-47.9588182845936
2        Plano Piloto  ...  -15.714424616292403,-47.9588182845936
3        Plano Piloto  ...  -15.714424616292403,-47.9588182845936
4        Plano Piloto  ...  -15.714424616292403,-47.9588182845936
...               ...  ...                                    ...
482332  Não Informado  ...                                nan,nan
482333  Não Informado  ...                                nan,nan
482334  Não Informado  ...                                nan,nan
482335  Não Informado  ...                                nan,nan
482336  Não Informado  ...                                nan,nan

[482337 rows x 9 columns]>

In [ ]:
# Salvando em .csv
df.to_csv('cov_df.csv')